In [1]:
!pip install transformers
!pip install tokenizers
!pip install tensorboard==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 33.1 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
     |████████████████████████████████| 6.6 MB 53.5 MB/s 
     |████████████████████████████████| 596 kB 54.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.8 MB 11.7 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that 

In [2]:
import pandas as pd
from tqdm import tqdm

In [3]:

import os
import getpass

#For a kaggle username & key, just go to your kaggle account and generate key
#The JSON file so downloaded contains both of them
if("examine-the-examiner.zip" not in os.listdir()):
  print("Copy these two values from the JSON file so generated")
  os.environ['KAGGLE_USERNAME'] = getpass.getpass(prompt='Kaggle username: ') 
  os.environ['KAGGLE_KEY'] =  getpass.getpass(prompt='Kaggle key: ')
  !kaggle datasets download -d therohk/examine-the-examiner
  !unzip /content/examine-the-examiner.zip

Copy these two values from the JSON file so generated
Kaggle username: ··········
Kaggle key: ··········
 75% 57.0M/75.7M [00:01<00:00, 38.5MB/s]
100% 75.7M/75.7M [00:01<00:00, 42.5MB/s]
Archive:  /content/examine-the-examiner.zip
  inflating: examiner-date-text.csv  


In [4]:
import regex as re
def basicPreprocess(text):
  try:
    processed_text = text.lower()
    processed_text = re.sub(r'\W +', ' ', processed_text)
  except Exception as e:
    print("Exception:",e,",on text:", text)
    return None
  return processed_text

In [5]:
import pandas as pd
from tqdm import tqdm

In [6]:
data = pd.read_csv("/content/examiner-date-text.csv")
print(data)

         publish_date                                      headline_text
0            20100101       100 Most Anticipated books releasing in 2010
1            20100101       10 best films of 2009 - What's on your list?
2            20100101  10 days of free admission at Lan Su Chinese Ga...
3            20100101      10 PlayStation games to watch out for in 2010
4            20100101  10 resolutions for a Happy New Year for you an...
...               ...                                                ...
3089776      20151231  Which is better investment, Lego bricks or gol...
3089777      20151231  Wild score three unanswered goals to defeat th...
3089778      20151231  With NASA and Russia on the sidelines, Europe ...
3089779      20151231  Wolf Pack battling opponents, officials on the...
3089780      20151231          Writespace hosts all genre open mic night

[3089781 rows x 2 columns]


In [7]:
data = data.sample(frac=1).sample(frac=1)
data = data[:200000]

In [8]:
print(data)

         publish_date                                      headline_text
1730786      20120315  Afghanistan: America shows the best and worst ...
2760271      20140518  Savannah police and animal control guide air-d...
2824465      20140815  Grief Awareness Day August 30, 2014; make it h...
984583       20110118  Xbox 360 Failure: A Problem Not Properly Dealt...
3028058      20150721  Can Sarah Palin make peace between Donald Trum...
...               ...                                                ...
2409529      20130605  Bynes job offer: Amanda Bynes turns down job a...
1353948      20110728  Latest details on WWE's plans for Alberto Del ...
1269253      20110608  Former St. Louis Ram Eric Crouch trying out fo...
2918465      20141230  Columnist's informal survey finds some women h...
410339       20100606  Free samples: Tea, One Touch diabetes monitor,...

[200000 rows x 2 columns]


In [9]:
data["headline_text"] = data["headline_text"].apply(basicPreprocess).dropna()

Exception: 'float' object has no attribute 'lower' ,on text: nan


In [10]:
print(data)

         publish_date                                      headline_text
1730786      20120315  afghanistan america shows the best and worst o...
2760271      20140518  savannah police and animal control guide air-d...
2824465      20140815  grief awareness day august 30 2014 make it hap...
984583       20110118  xbox 360 failure a problem not properly dealt ...
3028058      20150721  can sarah palin make peace between donald trum...
...               ...                                                ...
2409529      20130605  bynes job offer amanda bynes turns down job at...
1353948      20110728  latest details on wwe's plans for alberto del ...
1269253      20110608  former st louis ram eric crouch trying out for...
2918465      20141230  columnist's informal survey finds some women h...
410339       20100606  free samples tea one touch diabetes monitor mi...

[200000 rows x 2 columns]


In [11]:
data = data["headline_text"]
data = data.replace("\n"," ")

In [12]:
txt_files_dir = "/tmp/text_split"
!mkdir {txt_files_dir}

In [13]:

i=0
for row in tqdm(data.to_list()):
  file_name = os.path.join(txt_files_dir, str(i)+'.txt')
  try:
    f = open(file_name, 'w')
    f.write(row)
    f.close()
  except Exception as e:  #catch exceptions(for eg. empty rows)
    print(row, e) 
  i+=1

 19%|█▉        | 37769/200000 [00:03<00:17, 9167.48it/s]

nan write() argument must be str, not float


100%|██████████| 200000/200000 [00:17<00:00, 11411.07it/s]


In [14]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


paths = [str(x) for x in Path(txt_files_dir).glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

vocab_size=5000
# Customize training
tokenizer.train(files=paths, vocab_size=vocab_size, min_frequency=5, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [15]:
lm_data_dir = "/tmp/lm_data"
!mkdir {lm_data_dir}

In [16]:
train_split = 0.9
train_data_size = int(len(data)*train_split)

with open(os.path.join(lm_data_dir,'train.txt') , 'w') as f:
    for item in data[:train_data_size].tolist():
        f.write("%s\n" % item)

with open(os.path.join(lm_data_dir,'eval.txt') , 'w') as f:
    for item in data[train_data_size:].tolist():
        f.write("%s\n" % item)

In [17]:
!mkdir /content/models
!mkdir /content/models/smallBERTa

In [18]:
tokenizer.save_model("/content/models/smallBERTa", "smallBERTa")

['/content/models/smallBERTa/smallBERTa-vocab.json',
 '/content/models/smallBERTa/smallBERTa-merges.txt']

In [19]:
!mv /content/models/smallBERTa/smallBERTa-vocab.json /content/models/smallBERTa/vocab.json
!mv /content/models/smallBERTa/smallBERTa-merges.txt /content/models/smallBERTa/merges.txt

In [20]:

train_path = os.path.join(lm_data_dir,"train.txt")
eval_path = os.path.join(lm_data_dir,"eval.txt")

In [44]:
import json
config = {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "num_attention_heads": 1,
  "num_hidden_layers": 1,
  "vocab_size": vocab_size,
  "intermediate_size": 256,
  "max_position_embeddings": 256,
  "model_type": "roberta"
  }
with open("/content/models/smallBERTa/config.json", 'w') as fp:
    json.dump(config, fp)

In [30]:
%cd /content

!git clone https://github.com/huggingface/transformers/tree/v2.5.0

/content
Cloning into 'v2.5.0'...
fatal: repository 'https://github.com/huggingface/transformers/tree/v2.5.0/' not found


In [23]:
!nvidia-smi

Wed Jul  6 05:51:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
os.environ["train_path"] = train_path
os.environ["eval_path"] = eval_path
os.environ["CUDA_LAUNCH_BLOCKING"]='1'  #Makes for easier debugging (just in case)
weights_dir = "/content/models/smallBERTa/weights"
!mkdir {weights_dir}

In [50]:
cmd = '''python /content/transformers/examples/legacy/run_language_modeling.py --output_dir {0}  \
    --model_type roberta \
    --mlm \
    --train_data_file {1} \
    --eval_data_file {2} \
    --config_name /content/models/smallBERTa \
    --tokenizer_name /content/models/smallBERTa \
    --do_train \
    --line_by_line \
    --overwrite_output_dir \
    --do_eval \
    --block_size 256 \
    --learning_rate 1e-4 \
    --num_train_epochs 5 \
    --save_total_limit 2 \
    --save_steps 2000 \
    --logging_steps 500 \
    --per_gpu_eval_batch_size 32 \
    --per_gpu_train_batch_size 32 \
    '''.format(weights_dir, train_path, eval_path)


In [51]:

!{cmd}

07/06/2022 06:28:27 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
07/06/2022 06:28:27 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id